## Intro

Este codigo es parte del proceso de etiquetado.

Es la primera parte, donde se elige un modelo de MMsegmentation, se infieren varias imagenes, y con los resultados (una imagen segementada en X clases) se remapea para que la imagen tenga nuestras clases deseadas, consiguiendo así una primera instancia de etiquetas.

En este caso se uso el modelo ´san-vit-l14_coco-stuff164k-640x640.py´ y su checkpoint correspondiente https://download.openmmlab.com/mmsegmentation/v0.5/san/san-vit-l14_20230907-a11e098f.pth

Este modelo contaba con 172 clases que fueron reasignadas a nuestras 5.

In [ ]:
import os

!pip uninstall -y torch torchvision torchaudio numpy mmcv mmcv-full > /dev/null 2>&1
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118 > /dev/null 2>&1
!pip3 install ftfy > /dev/null 2>&1
!pip3 install openmim > /dev/null 2>&1
!mim install mmengine > /dev/null 2>&1
!mim install "mmcv==2.1.0" > /dev/null 2>&1

!git clone https://github.com/open-mmlab/mmsegmentation.git > /dev/null 2>&1
%cd mmsegmentation
!git checkout main > /dev/null 2>&1
!pip install -e . > /dev/null 2>&1

!pip install numpy==1.23.5 --force-reinstall > /dev/null 2>&1

os.kill(os.getpid(), 9)

/content/mmsegmentation


Importante montar el drive y tener las images a inferir allí.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Definicion de Funciones y de remapeo.


In [ ]:
classes = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
    'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
    'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
    'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
    'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
    'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
    'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
    'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
    'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
    'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'banner',
    'blanket', 'branch', 'bridge', 'building-other', 'bush', 'cabinet',
    'cage', 'cardboard', 'carpet', 'ceiling-other', 'ceiling-tile',
    'cloth', 'clothes', 'clouds', 'counter', 'cupboard', 'curtain',
    'desk-stuff', 'dirt', 'door-stuff', 'fence', 'floor-marble',
    'floor-other', 'floor-stone', 'floor-tile', 'floor-wood', 'flower',
    'fog', 'food-other', 'fruit', 'furniture-other', 'grass', 'gravel',
    'ground-other', 'hill', 'house', 'leaves', 'light', 'mat', 'metal',
    'mirror-stuff', 'moss', 'mountain', 'mud', 'napkin', 'net', 'paper',
    'pavement', 'pillow', 'plant-other', 'plastic', 'platform',
    'playingfield', 'railing', 'railroad', 'river', 'road', 'rock', 'roof',
    'rug', 'salad', 'sand', 'sea', 'shelf', 'sky-other', 'skyscraper',
    'snow', 'solid-other', 'stairs', 'stone', 'straw', 'structural-other',
    'table', 'tent', 'textile-other', 'towel', 'tree', 'vegetable',
    'wall-brick', 'wall-concrete', 'wall-other', 'wall-panel',
    'wall-stone', 'wall-tile', 'wall-wood', 'water-other', 'waterdrops',
    'window-blind', 'window-other', 'wood'
]

# Mapping function
def remap_classes(classes):
    remapped = {}
    for idx, cls in enumerate(classes):
        if cls in ['road', 'dirt', 'grass', 'pavement', 'floor-marble', 'floor-other', 'floor-stone', 'floor-tile', 'floor-wood']:
            remapped[idx] = 0  # road
        elif cls in ['tree', 'apple', 'bush', 'flower', 'leaves', 'moss', 'vegetable', 'plant-other']:
            remapped[idx] = 1  # vegetation
        elif cls in ['sky-other', 'clouds', 'fog', 'hill']:
            remapped[idx] = 2  # sky
        elif cls in ['mud', 'person', 'rock', 'building-other', 'furniture-other', 'wall-other',
                     'fence', 'gravel', 'net', 'river', 'sand', 'sea', 'stone', 'wood']:
            remapped[idx] = 3  # obstacle
        else:
            remapped[idx] = 4  # other
    return remapped

def remap_segmentation_mask(predictions, remapped_classes):
    # Create a new array with the same shape as the predictions
    remapped_mask = np.zeros_like(predictions, dtype=np.uint8)

    # Iterate through the mapping and remap the indices
    for original_idx, new_idx in remapped_classes.items():
        remapped_mask[predictions == original_idx] = new_idx

    return remapped_mask

def save_segmentation_mask(seg_mask, original_img_path, output_dir):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Convert mask to uint8 grayscale if not already
    seg_mask = seg_mask.astype(np.uint8)

    # Get image name without extension
    base_name = os.path.splitext(os.path.basename(original_img_path))[0]

    # Define output path
    save_path = os.path.join(output_dir, f"{base_name}.png")

    # Save the mask (grayscale)
    cv2.imwrite(save_path, seg_mask)

def visualize_and_save_segmentation(img_path, seg_mask, output_dir=None, alpha=0.5):
    from IPython import display

    # Load original image
    img = mmcv.imread(img_path, channel_order='rgb')
    seg_mask = seg_mask.astype(np.uint8)
    img = mmcv.imresize(img, (seg_mask.shape[1], seg_mask.shape[0]))

    # Define your custom color palette and class names
    color_palette = {
        0: ((128, 64, 128), 'road'),
        1: ((0, 128, 0), 'vegetation'),
        2: ((70, 130, 180), 'sky'),
        3: ((220, 20, 60), 'obstacle'),
        4: ((128, 128, 128), 'other')
    }

    # Create a blank color mask
    color_mask = np.zeros((seg_mask.shape[0], seg_mask.shape[1], 3), dtype=np.uint8)

    for class_id, (color, _) in color_palette.items():
        color_mask[seg_mask == class_id] = color

    # Blend original image with color mask
    blended = cv2.addWeighted(img, 1 - alpha, color_mask, alpha, 0)

    # Add class labels
    for class_id, (color, label) in color_palette.items():
        mask = (seg_mask == class_id).astype(np.uint8)
        if np.any(mask):
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                M = cv2.moments(largest_contour)
                if M["m00"] > 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    cv2.putText(blended, label, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX,
                                0.7, (255, 255, 255), 2, cv2.LINE_AA)

    # Display the image
    plt.imshow(blended)
    plt.axis('off')
    plt.show()
    plt.pause(0.5)
    plt.clf()
    display.clear_output(wait=True)

    # Save the image if output_dir is provided
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        save_path = os.path.join(output_dir, f"{base_name}_vis.png")
        cv2.imwrite(save_path, cv2.cvtColor(blended, cv2.COLOR_RGB2BGR))  # Convert to BGR for OpenCV saving

In [ ]:
!cp -r "/content/drive/MyDrive/Facultad/Onceavo semestre/TSCF/Imgs/RobotInTheMud/RobotInTheMud" /content
#!unzip /content/RobotInTheMud.zip -d /content

In [ ]:
from mmseg.apis import inference_model, init_model
from mmseg.utils import register_all_modules
import mmcv
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

imgDir = '/content/RobotInTheMud' # Imagenes a generar etiquetas
output_mask_dir = "/content/segmentation_outputs" # Donde guardar etiquetas
output_vis_dir = "/content/visualizations" # Donde guardar visualizacion para poder ver resultado.

remapped_classes = remap_classes(classes)

model = init_model(
    config='/content/mmsegmentation/configs/san/san-vit-l14_coco-stuff164k-640x640.py',
    checkpoint='https://download.openmmlab.com/mmsegmentation/v0.5/san/san-vit-l14_20230907-a11e098f.pth',
    device='cuda:0'
)

img_files = [os.path.join(imgDir, f) for f in os.listdir(imgDir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Inference loop
for img_path in img_files:
    result = inference_model(model, img_path)
    pred_mask = result.pred_sem_seg.data.squeeze().cpu().numpy()
    remapped_mask = remap_segmentation_mask(pred_mask, remapped_classes)
    save_segmentation_mask(remapped_mask, img_path, output_mask_dir)
    visualize_and_save_segmentation(img_path, remapped_mask, output_vis_dir)



<Figure size 640x480 with 0 Axes>

Las imagenes se generan en el colab, si las quiero descargar, copiarlas a mi drive y descargarlas.


In [ ]:
!cp -r /content/segmentation_outputs "/content/drive/MyDrive/Facultad/Onceavo semestre/TSCF/Temp/24 5 2025"